# RTP Forecasting

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

2.12.1


In [3]:
cwd = os.getcwd()

In [4]:
def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)

In [5]:
model_path = os.path.join(cwd,'saved_model')
make_dir(model_path)

In [6]:
#timing callback
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [7]:
# Plot history and future
def plot_predictions(pred , actual, title):
    plt.figure(figsize=(20, 4), dpi=150)
    plt.plot(np.arange(len(pred)), np.array(pred),label='cnn',alpha=0.7)
    plt.plot(np.arange(len(pred)), np.array(actual),label='PF', alpha=0.7)
    plt.axhline(y=0, color='black', linestyle='--', lw=1, alpha=0.5)
    plt.legend(loc='upper right')
    plt.xlabel('Time step' ,  fontsize=18)
    plt.ylabel('Price' , fontsize=18)
    plt.title(title, fontsize=16)

In [8]:
# Plot history and future
def plot_predictions_slide(pred_1,pred_2,pred_3, actual, title):
    plt.figure(figsize=(20, 4), dpi=150)
    plt.plot(np.arange(len(pred_1)), np.array(actual),label='PF', alpha=0.7)
    plt.plot(np.arange(len(pred_1)), np.array(pred_1),label='cnn-24',alpha=0.7)
    plt.plot(np.arange(len(pred_1)), np.array(pred_2),label='cnn-48',alpha=0.7)
    plt.plot(np.arange(len(pred_1)), np.array(pred_3),label='cnn-27',alpha=0.7)
    plt.axhline(y=0, color='black', linestyle='--', lw=1, alpha=0.5)
    plt.legend(loc='upper right')
    plt.xlabel('Time step' ,  fontsize=18)
    plt.ylabel('Price' , fontsize=18)
    plt.title(title, fontsize=16)

In [9]:
zones = ['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL',
        'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 'WEST']
# zone = 'CAPITL'
year = 2021

In [21]:
# Read each timeseries (RTP = Real-Time Price, DAP = Day-Ahead Price, LF = Load Forecast)
raw_DAP = pd.read_csv("nyiso/da_lmp_zones_df_2015_2021.csv", index_col=0)
raw_RTP = pd.read_csv("nyiso/rt_lmp_zones_df_2015_2021.csv", index_col=0)
raw_LF = pd.read_csv("nyiso/load_frcstd_df_2015_2021.csv", index_col=0)

# Prepare the dataset as a dataframe
raw_data = pd.concat([raw_DAP.loc[:,zone], raw_LF.loc[:,zone], raw_RTP.loc[:,zone]],
                       axis=1).loc['2017-01-01 05:00:00+00:00':]
raw_data.columns = ['DAP', 'LF', 'RTP']
raw_data.index.names = ['date']
raw_data.to_csv('nyiso/NYISO_'+zone+'_raw.csv')

log_data = raw_data.copy(deep=True)
log_data.loc[:,"DAP"] = np.log(raw_data.loc[:,"DAP"] + 1 - min(raw_data.loc[:,"DAP"]))
log_data.loc[:,"RTP"] = np.log(raw_data.loc[:,"RTP"] + 1 - min(raw_data.loc[:,"RTP"]))
log_data.to_csv('nyiso/NYISO_'+zone+'_log.csv')

# Split dataset: 2015 year for training and 2016-2017 years for testing
x_train_df = log_data.iloc[:8760*4+24,:]
x_test_df = log_data.iloc[8760*4+24:,:]

y_train_df = log_data.iloc[:8760*4+24,2:]
y_test_df = log_data.iloc[8760*4+24:,2:]

# Standardization
x_mean, x_std = x_train_df.mean(), x_train_df.std()
y_mean, y_std = y_train_df.mean(), y_train_df.std()

x_train = ((x_train_df - x_mean)/x_std).to_numpy()
x_test = ((x_test_df - x_mean)/x_std).to_numpy()

y_train = ((y_train_df - y_mean)/y_std).to_numpy()
y_test = ((y_test_df - y_mean)/y_std).to_numpy()

############## Lag = 24 ###############

n_steps_in = 48
n_steps_out = 24

x_train_cnn = np.array([x_train[i:i+n_steps_in] for i in range(0, x_train.shape[0]-n_steps_in-n_steps_out+1)])
y_train_cnn = np.array([y_train[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train.shape[0]-n_steps_in-n_steps_out+1)])

x_test_cnn = np.array([x_test[i:i+n_steps_in] for i in range(0, x_test.shape[0]-n_steps_in-n_steps_out+1)])
y_test_cnn = np.array([y_test[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test.shape[0]-n_steps_in-n_steps_out+1)])

y_test_cnn = np.array([y_test[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test.shape[0]-n_steps_in-n_steps_out+1)])

cnn_model = load_model(os.path.join(model_path,'cnn_model_CAPITL_48h.h5'))
y_test_pred = cnn_model.predict(x_test_cnn)

# Evaluation metrics
print(zone)
print('24h MAE: {:.4f}'.format(np.abs(y_test_pred - y_test_cnn[:,:,0]).mean()))
print('')

y_test_pred_rescale = y_test_pred*y_std.values + y_mean.values
y_test_cnn_rescale = y_test_cnn*y_std.values + y_mean.values
y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(raw_data.loc[:,"RTP"])
y_test_cnn_invlog = np.exp(y_test_cnn_rescale) -1 + min(raw_data.loc[:,"RTP"])

# Evaluation metrics

print(zone)
print('24h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog - y_test_cnn_invlog[:,:,0]).mean()))
print('')

272/272 [==============================] - 1s 2ms/step
CAPITL
24h MAE: 0.4724

CAPITL
24h MAE: 11.9008



In [18]:
cnn_model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_50 (Conv1D)          (None, 24, 32)            320       
                                                                 
 conv1d_51 (Conv1D)          (None, 12, 32)            3104      
                                                                 
 max_pooling1d_25 (MaxPoolin  (None, 12, 32)           0         
 g1D)                                                            
                                                                 
 flatten_25 (Flatten)        (None, 384)               0         
                                                                 
 dense_50 (Dense)            (None, 64)                24640     
                                                                 
 dense_51 (Dense)            (None, 24)                1560      
                                                     

In [26]:
# set hyperparameters
n_filters  = 32  # number of filters
n_neurons  = 64  # number of neurons in the Dense layer
activation     = 'relu' # activation function
kernel_size    = 3
pool_size = 1
learning_rate  = 0.0001
minibatch_size = 32
num_epochs     = 50
    
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

cnn_model = load_model(os.path.join(model_path,'cnn_model_N.Y.C._48h.h5'))

cnn_model.get_layer(index=0).trainable = False
cnn_model.get_layer(index=1).trainable = False
cnn_model.get_layer(index=2).trainable = False
cnn_model.get_layer(index=3).trainable = False
cnn_model.get_layer(index=4).trainable = False
cnn_model.get_layer(index=5).trainable = True
    
history = cnn_model.fit(x_train_cnn, y_train_cnn, 
                    batch_size      = minibatch_size,
                    epochs          = num_epochs,
                    validation_split= 0.2, 
                    verbose         = 1,
                    callbacks       = [early_stop])

Epoch 1/50
875/875 [==============================] - 6s 6ms/step - loss: 0.5288 - val_loss: 0.2046
Epoch 2/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5253 - val_loss: 0.2075
Epoch 3/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5238 - val_loss: 0.2052
Epoch 4/50
875/875 [==============================] - 6s 6ms/step - loss: 0.5223 - val_loss: 0.2045
Epoch 5/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5208 - val_loss: 0.2050
Epoch 6/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5189 - val_loss: 0.2065
Epoch 7/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5175 - val_loss: 0.2075
Epoch 8/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5157 - val_loss: 0.2072
Epoch 9/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5140 - val_loss: 0.2053
Epoch 10/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5128 - val_loss: 0.2071

In [27]:
y_test_pred = cnn_model.predict(x_test_cnn)

# Evaluation metrics
print(zone)
print('24h MAE: {:.4f}'.format(np.abs(y_test_pred - y_test_cnn[:,:,0]).mean()))
print('')

y_test_pred_rescale = y_test_pred*y_std.values + y_mean.values
y_test_cnn_rescale = y_test_cnn*y_std.values + y_mean.values
y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(raw_data.loc[:,"RTP"])
y_test_cnn_invlog = np.exp(y_test_cnn_rescale) -1 + min(raw_data.loc[:,"RTP"])

# Evaluation metrics

print(zone)
print('24h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog - y_test_cnn_invlog[:,:,0]).mean()))
print('')

272/272 [==============================] - 1s 2ms/step
CAPITL
24h MAE: 0.5016

CAPITL
24h MAE: 12.6220



In [28]:
y_test_pred = cnn_model.predict(x_test_cnn)

# Evaluation metrics
print(zone)
print('24h MAE: {:.4f}'.format(np.abs(y_test_pred - y_test_cnn[:,:,0]).mean()))
print('')

y_test_pred_rescale = y_test_pred*y_std.values + y_mean.values
y_test_cnn_rescale = y_test_cnn*y_std.values + y_mean.values
y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(raw_data.loc[:,"RTP"])
y_test_cnn_invlog = np.exp(y_test_cnn_rescale) -1 + min(raw_data.loc[:,"RTP"])

# Evaluation metrics

print(zone)
print('24h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog - y_test_cnn_invlog[:,:,0]).mean()))
print('')

273/273 [==============================] - 1s 2ms/step
CAPITL
24h MAE: 0.4999

CAPITL
24h MAE: 12.6152



### Data Import

In [11]:
for zone in zones:
    # Read each timeseries (RTP = Real-Time Price, DAP = Day-Ahead Price, LF = Load Forecast)
    raw_DAP = pd.read_csv("nyiso/da_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_RTP = pd.read_csv("nyiso/rt_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_LF = pd.read_csv("nyiso/load_frcstd_df_2015_2021.csv", index_col=0)
    
    # Prepare the dataset as a dataframe
    raw_data = pd.concat([raw_DAP.loc[:,zone], raw_LF.loc[:,zone], raw_RTP.loc[:,zone]],
                           axis=1).loc['2017-01-01 05:00:00+00:00':]
    raw_data.columns = ['DAP', 'LF', 'RTP']
    raw_data.index.names = ['date']
    raw_data.to_csv('nyiso/NYISO_'+zone+'_raw.csv')

    log_data = raw_data.copy(deep=True)
    log_data.loc[:,"DAP"] = np.log(raw_data.loc[:,"DAP"] + 1 - min(raw_data.loc[:,"DAP"]))
    log_data.loc[:,"RTP"] = np.log(raw_data.loc[:,"RTP"] + 1 - min(raw_data.loc[:,"RTP"]))
    log_data.to_csv('nyiso/NYISO_'+zone+'_log.csv')

    # Split dataset: 2015 year for training and 2016-2017 years for testing
    x_train_df = log_data.iloc[8760*3:8760*4+24,:]
    x_test_df = log_data.iloc[8760*4+24:,:]

    y_train_df = log_data.iloc[8760*3:8760*4+24,2:]
    y_test_df = log_data.iloc[8760*4+24:,2:]

    # Standardization
    x_mean, x_std = x_train_df.mean(), x_train_df.std()
    y_mean, y_std = y_train_df.mean(), y_train_df.std()

    x_train = ((x_train_df - x_mean)/x_std).to_numpy()
    x_test = ((x_test_df - x_mean)/x_std).to_numpy()

    y_train = ((y_train_df - y_mean)/y_std).to_numpy()
    y_test = ((y_test_df - y_mean)/y_std).to_numpy()

    ############## Lag = 48 ###############

    n_steps_in = 48
    n_steps_out = 24

    x_train_cnn = np.array([x_train[i:i+n_steps_in] for i in range(0, x_train.shape[0]-n_steps_in-n_steps_out+1)])
    y_train_cnn = np.array([y_train[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train.shape[0]-n_steps_in-n_steps_out+1)])

    x_test_cnn = np.array([x_test[i:i+n_steps_in] for i in range(0, x_test.shape[0]-n_steps_in-n_steps_out+1)])
    y_test_cnn = np.array([y_test[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test.shape[0]-n_steps_in-n_steps_out+1)])

    print(x_train_cnn.shape,y_train_cnn.shape,x_test_cnn.shape,y_test_cnn.shape)
    
    # set hyperparameters
    n_filters  = 32  # number of filters
    n_neurons  = 64  # number of neurons in the Dense layer
    activation     = 'relu' # activation function
    kernel_size    = 3
    pool_size = 1
    learning_rate  = 0.0001
    minibatch_size = 32
    num_epochs     = 10

    cnn_model = load_model(os.path.join(model_path,'cnn_model_N.Y.C._48h.h5'))

    y_test_pred = cnn_model.predict(x_test_cnn)

    # Evaluation metrics
    print(zone)
    print('Pre-trained:')
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred - y_test_cnn[:,:,0]).mean()))

    y_test_pred_rescale = y_test_pred*y_std.values + y_mean.values
    y_test_cnn_rescale = y_test_cnn*y_std.values + y_mean.values
    y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(raw_data.loc[:,"RTP"])
    y_test_cnn_invlog = np.exp(y_test_cnn_rescale) -1 + min(raw_data.loc[:,"RTP"])

    # Evaluation metrics
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog - y_test_cnn_invlog[:,:,0]).mean()))
    print('')
    
    cnn_model.get_layer(index=0).trainable = False
    cnn_model.get_layer(index=1).trainable = False
    cnn_model.get_layer(index=2).trainable = False
    cnn_model.get_layer(index=3).trainable = False
    cnn_model.get_layer(index=4).trainable = False
    cnn_model.get_layer(index=5).trainable = True

    # Running training

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

    history = cnn_model.fit(x_train_cnn, y_train_cnn, 
                            batch_size = minibatch_size,
                            epochs = num_epochs,
                            validation_split=0.2, verbose=1,
                            callbacks=[early_stop],
                            shuffle=False)

    # Saving the model
    cnn_model.save(os.path.join(model_path,'TL_1y_cnn_model_'+zone+'_48h.h5'))

    y_test_pred = cnn_model.predict(x_test_cnn)

    # Evaluation metrics
    print(zone)
    print('Re-trained:')
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred - y_test_cnn[:,:,0]).mean()))

    y_test_pred_rescale = y_test_pred*y_std.values + y_mean.values
    y_test_cnn_rescale = y_test_cnn*y_std.values + y_mean.values
    y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(raw_data.loc[:,"RTP"])
    y_test_cnn_invlog = np.exp(y_test_cnn_rescale) -1 + min(raw_data.loc[:,"RTP"])

    # Evaluation metrics
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog - y_test_cnn_invlog[:,:,0]).mean()))
    print('')

(8713, 48, 3) (8713, 24, 1) (8689, 48, 3) (8689, 24, 1)
272/272 [==============================] - 1s 3ms/step
CAPITL
Pre-trained:
24h MAE: 0.8094
24h MAE: 12.5288

Epoch 1/10
218/218 [==============================] - 2s 8ms/step - loss: 0.3555 - val_loss: 1.2090
Epoch 2/10
218/218 [==============================] - 1s 7ms/step - loss: 0.3514 - val_loss: 1.2168
Epoch 3/10
218/218 [==============================] - 2s 7ms/step - loss: 0.3492 - val_loss: 1.2217
Epoch 4/10
218/218 [==============================] - 1s 7ms/step - loss: 0.3474 - val_loss: 1.2264
Epoch 5/10
218/218 [==============================] - 1s 7ms/step - loss: 0.3458 - val_loss: 1.2307
Epoch 6/10
218/218 [==============================] - 1s 7ms/step - loss: 0.3444 - val_loss: 1.2350
Epoch 7/10
218/218 [==============================] - 2s 7ms/step - loss: 0.3431 - val_loss: 1.2393
Epoch 8/10
218/218 [==============================] - 2s 7ms/step - loss: 0.3419 - val_loss: 1.2437
Epoch 9/10
218/218 [===============

In [ ]:
for zone in zones:
    # Read each timeseries (RTP = Real-Time Price, DAP = Day-Ahead Price, LF = Load Forecast)
    raw_DAP = pd.read_csv("nyiso/da_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_RTP = pd.read_csv("nyiso/rt_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_LF = pd.read_csv("nyiso/load_frcstd_df_2015_2021.csv", index_col=0)
    
    # Prepare the dataset as a dataframe
    raw_data = pd.concat([raw_DAP.loc[:,zone], raw_LF.loc[:,zone], raw_RTP.loc[:,zone]],
                           axis=1).loc['2017-01-01 05:00:00+00:00':]
    raw_data.columns = ['DAP', 'LF', 'RTP']
    raw_data.index.names = ['date']
    raw_data.to_csv('nyiso/NYISO_'+zone+'_raw.csv')

    log_data = raw_data.copy(deep=True)
    log_data.loc[:,"DAP"] = np.log(raw_data.loc[:,"DAP"] + 1 - min(raw_data.loc[:,"DAP"]))
    log_data.loc[:,"RTP"] = np.log(raw_data.loc[:,"RTP"] + 1 - min(raw_data.loc[:,"RTP"]))
    log_data.to_csv('nyiso/NYISO_'+zone+'_log.csv')

    # Split dataset: 2015 year for training and 2016-2017 years for testing
    x_train_df = log_data.iloc[:8760*4+24,:]
    x_test_df = log_data.iloc[8760*4+24:,:]

    y_train_df = log_data.iloc[:8760*4+24,2:]
    y_test_df = log_data.iloc[8760*4+24:,2:]

    # Standardization
    x_mean, x_std = x_train_df.mean(), x_train_df.std()
    y_mean, y_std = y_train_df.mean(), y_train_df.std()

    x_train = ((x_train_df - x_mean)/x_std).to_numpy()
    x_test = ((x_test_df - x_mean)/x_std).to_numpy()

    y_train = ((y_train_df - y_mean)/y_std).to_numpy()
    y_test = ((y_test_df - y_mean)/y_std).to_numpy()

    ############## Lag = 48 ###############

    n_steps_in = 48
    n_steps_out = 24

    x_train_cnn = np.array([x_train[i:i+n_steps_in] for i in range(0, x_train.shape[0]-n_steps_in-n_steps_out+1)])
    y_train_cnn = np.array([y_train[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train.shape[0]-n_steps_in-n_steps_out+1)])

    x_test_cnn = np.array([x_test[i:i+n_steps_in] for i in range(0, x_test.shape[0]-n_steps_in-n_steps_out+1)])
    y_test_cnn = np.array([y_test[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test.shape[0]-n_steps_in-n_steps_out+1)])

    print(x_train_cnn.shape,y_train_cnn.shape,x_test_cnn.shape,y_test_cnn.shape)
    
    # set hyperparameters
    n_filters  = 32  # number of filters
    n_neurons  = 64  # number of neurons in the Dense layer
    activation     = 'relu' # activation function
    kernel_size    = 3
    pool_size = 1
    learning_rate  = 0.0001
    minibatch_size = 32
    num_epochs     = 50

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

    cnn_model = load_model(os.path.join(model_path,'cnn_model_N.Y.C._48h.h5'))

    y_test_pred = cnn_model.predict(x_test_cnn)

    # Evaluation metrics
    print(zone)
    print('Pre-trained:')
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred - y_test_cnn[:,:,0]).mean()))

    y_test_pred_rescale = y_test_pred*y_std.values + y_mean.values
    y_test_cnn_rescale = y_test_cnn*y_std.values + y_mean.values
    y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(raw_data.loc[:,"RTP"])
    y_test_cnn_invlog = np.exp(y_test_cnn_rescale) -1 + min(raw_data.loc[:,"RTP"])

    # Evaluation metrics
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog - y_test_cnn_invlog[:,:,0]).mean()))
    print('')
    
    cnn_model.get_layer(index=0).trainable = False
    cnn_model.get_layer(index=1).trainable = False
    cnn_model.get_layer(index=2).trainable = False
    cnn_model.get_layer(index=3).trainable = False
    cnn_model.get_layer(index=4).trainable = False
    cnn_model.get_layer(index=5).trainable = True

    history = cnn_model.fit(x_train_cnn, y_train_cnn, 
                        batch_size      = minibatch_size,
                        epochs          = num_epochs,
                        validation_split= 0.2, 
                        verbose         = 1,
                        callbacks       = [early_stop])

    # Running training

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

    history = cnn_model.fit(x_train_cnn, y_train_cnn, 
                            batch_size = minibatch_size,
                            epochs = num_epochs,
                            validation_split=0.2, verbose=1,
                            callbacks=[early_stop],
                            shuffle=False)

    # Saving the model
    cnn_model.save(os.path.join(model_path,'TL_50ep_cnn_model_'+zone+'_48h.h5'))

    y_test_pred = cnn_model.predict(x_test_cnn)

    # Evaluation metrics
    print(zone)
    print('Re-trained:')
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred - y_test_cnn[:,:,0]).mean()))

    y_test_pred_rescale = y_test_pred*y_std.values + y_mean.values
    y_test_cnn_rescale = y_test_cnn*y_std.values + y_mean.values
    y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(raw_data.loc[:,"RTP"])
    y_test_cnn_invlog = np.exp(y_test_cnn_rescale) -1 + min(raw_data.loc[:,"RTP"])

    # Evaluation metrics
    print('24h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog - y_test_cnn_invlog[:,:,0]).mean()))
    print('')

(34993, 48, 3) (34993, 24, 1) (8689, 48, 3) (8689, 24, 1)
272/272 [==============================] - 1s 2ms/step
CAPITL
Pre-trained:
24h MAE: 0.4836
24h MAE: 12.1424

Epoch 1/50
875/875 [==============================] - 6s 6ms/step - loss: 0.5501 - val_loss: 0.1993
Epoch 2/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5421 - val_loss: 0.2003
Epoch 3/50
875/875 [==============================] - 5s 5ms/step - loss: 0.5384 - val_loss: 0.2002
Epoch 4/50
875/875 [==============================] - 6s 7ms/step - loss: 0.5354 - val_loss: 0.2001
Epoch 5/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5330 - val_loss: 0.2013
Epoch 6/50
875/875 [==============================] - 5s 5ms/step - loss: 0.5305 - val_loss: 0.2018
Epoch 7/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5281 - val_loss: 0.2016
Epoch 8/50
875/875 [==============================] - 5s 6ms/step - loss: 0.5264 - val_loss: 0.2023
Epoch 9/50
875/875 [=============

In [13]:
for zone in zones:
    # Read each timeseries (RTP = Real-Time Price, DAP = Day-Ahead Price, LF = Load Forecast)
    raw_DAP = pd.read_csv("nyiso/da_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_RTP = pd.read_csv("nyiso/rt_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_LF = pd.read_csv("nyiso/load_frcstd_df_2015_2021.csv", index_col=0)
    
    # Prepare the dataset as a dataframe
    raw_data = pd.concat([raw_DAP.loc[:,zone], raw_LF.loc[:,zone], raw_RTP.loc[:,zone]],
                           axis=1).loc['2017-01-01 05:00:00+00:00':]
    raw_data.columns = ['DAP', 'LF', 'RTP']
    raw_data.index.names = ['date']
    raw_data.to_csv('nyiso/NYISO_'+zone+'_raw.csv')

    log_data = raw_data.copy(deep=True)
    log_data.loc[:,"DAP"] = np.log(raw_data.loc[:,"DAP"] + 1 - min(raw_data.loc[:,"DAP"]))
    log_data.loc[:,"RTP"] = np.log(raw_data.loc[:,"RTP"] + 1 - min(raw_data.loc[:,"RTP"]))
    log_data.to_csv('nyiso/NYISO_'+zone+'_log.csv')

    # Split dataset: 2015 year for training and 2016-2017 years for testing
    x_train_df = log_data.iloc[8760*3:8760*4+24,:]
    x_test_df = log_data.iloc[8760*4+24:,:]

    y_train_df = log_data.iloc[8760*3:8760*4+24,2:]
    y_test_df = log_data.iloc[8760*4+24:,2:]

    # Standardization
    x_mean, x_std = x_train_df.mean(), x_train_df.std()
    y_mean, y_std = y_train_df.mean(), y_train_df.std()

    x_train = ((x_train_df - x_mean)/x_std).to_numpy()
    x_test = ((x_test_df - x_mean)/x_std).to_numpy()

    y_train = ((y_train_df - y_mean)/y_std).to_numpy()
    y_test = ((y_test_df - y_mean)/y_std).to_numpy()

    
    ############## Lag = 48 ###############

    n_steps_in = 48
    n_steps_out = 24

    x_train_cnn = np.array([x_train[i:i+n_steps_in] for i in range(0, x_train.shape[0]-n_steps_in-n_steps_out+1)])
    y_train_cnn = np.array([y_train[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train.shape[0]-n_steps_in-n_steps_out+1)])

    x_test_cnn = np.array([x_test[i:i+n_steps_in] for i in range(0, x_test.shape[0]-n_steps_in-n_steps_out+1)])
    y_test_cnn = np.array([y_test[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test.shape[0]-n_steps_in-n_steps_out+1)])

    print(x_train_cnn.shape,y_train_cnn.shape,x_test_cnn.shape,y_test_cnn.shape)

    # set hyperparameters
    n_filters  = 32  # number of filters
    n_neurons  = 64  # number of neurons in the Dense layer
    activation     = 'relu' # activation function
    kernel_size    = 3
    pool_size = 1
    learning_rate  = 0.0001
    minibatch_size = 32
    num_epochs     = 50

    # Building the model
    # define model
    cnn_model = Sequential()
    cnn_model.add(Conv1D(filters=n_filters,kernel_size=kernel_size, strides=2, padding='same',
                         input_shape=(x_train_cnn.shape[1],x_train_cnn.shape[2]), activation=activation))
    cnn_model.add(Conv1D(filters=n_filters,kernel_size=kernel_size, strides=2, padding='same',
                         input_shape=(x_train_cnn.shape[1],x_train_cnn.shape[2]), activation=activation))
    cnn_model.add(MaxPooling1D(pool_size=pool_size))
    cnn_model.add(Flatten())
    cnn_model.add(Dense(n_neurons, activation=activation))
    cnn_model.add(Dense(n_steps_out, activation='linear'))
    cnn_model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

    # Running training

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

    history = cnn_model.fit(x_train_cnn, y_train_cnn, 
                            batch_size = minibatch_size,
                            epochs = num_epochs,
                            validation_split=0.2, verbose=1,
                            callbacks=[early_stop],
                            shuffle=False)

    # Saving the model
    model_path = os.path.join(cwd,'saved_model')
    make_dir(model_path)
    cnn_model.save(os.path.join(model_path,'1y_cnn_model_'+zone+'_48h.h5'))

    ################# Evaluation ##################

    n_steps_in = 48
    x_test_cnn = np.array([x_test[i:i+n_steps_in] for i in range(0, x_test.shape[0]-n_steps_in-n_steps_out+1)])
    y_test_cnn_48 = np.array([y_test[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test.shape[0]-n_steps_in-n_steps_out+1)])
    cnn_model = load_model(os.path.join(model_path,'1y_cnn_model_'+zone+'_48h.h5'))
    y_test_pred_48 = cnn_model.predict(x_test_cnn)

    # Evaluation metrics
    print(zone)
    print('48h MAE: {:.4f}'.format(np.abs(y_test_pred_48 - y_test_cnn_48[:,:,0]).mean()))

    y_test_pred_rescale_48 = y_test_pred_48*y_std.values + y_mean.values
    y_test_cnn_rescale_48 = y_test_cnn_48*y_std.values + y_mean.values
    y_test_pred_invlog_48 = np.exp(y_test_pred_rescale_48) -1 + min(raw_data.loc[:,"RTP"])
    y_test_cnn_invlog_48 = np.exp(y_test_cnn_rescale_48) -1 + min(raw_data.loc[:,"RTP"])

    # Evaluation metrics

    print('48h MAE: {:.4f}'.format(np.abs(y_test_pred_invlog_48 - y_test_cnn_invlog_48[:,:,0]).mean()))
    print('')

(8713, 48, 3) (8713, 24, 1) (8689, 48, 3) (8689, 24, 1)
Epoch 1/50
218/218 [==============================] - 2s 7ms/step - loss: 0.5305 - val_loss: 2.6437
Epoch 2/50
218/218 [==============================] - 1s 6ms/step - loss: 0.4648 - val_loss: 2.4213
Epoch 3/50
218/218 [==============================] - 1s 6ms/step - loss: 0.4339 - val_loss: 2.2216
Epoch 4/50
218/218 [==============================] - 1s 6ms/step - loss: 0.4164 - val_loss: 2.0476
Epoch 5/50
218/218 [==============================] - 1s 7ms/step - loss: 0.4024 - val_loss: 1.8831
Epoch 6/50
218/218 [==============================] - 2s 7ms/step - loss: 0.3909 - val_loss: 1.7480
Epoch 7/50
218/218 [==============================] - 1s 6ms/step - loss: 0.3823 - val_loss: 1.6526
Epoch 8/50
218/218 [==============================] - 1s 6ms/step - loss: 0.3758 - val_loss: 1.5876
Epoch 9/50
218/218 [==============================] - 1s 6ms/step - loss: 0.3710 - val_loss: 1.5407
Epoch 10/50
218/218 [=======================

In [22]:
for zone in zones:
    # Read each timeseries (RTP = Real-Time Price, DAP = Day-Ahead Price, LF = Load Forecast)
    raw_DAP = pd.read_csv("nyiso/da_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_RTP = pd.read_csv("nyiso/rt_lmp_zones_df_2015_2021.csv", index_col=0)
    raw_LF = pd.read_csv("nyiso/load_frcstd_df_2015_2021.csv", index_col=0)
    
    # Prepare the dataset as a dataframe
    raw_data = pd.concat([raw_DAP.loc[:,zone], raw_LF.loc[:,zone], raw_RTP.loc[:,zone]],
                           axis=1).loc['2017-01-01 05:00:00+00:00':]
    raw_data.columns = ['DAP', 'LF', 'RTP']
    raw_data.index.names = ['date']
    raw_data.to_csv('nyiso/NYISO_'+zone+'_raw.csv')

    log_data = raw_data.copy(deep=True)
    log_data.loc[:,"DAP"] = np.log(raw_data.loc[:,"DAP"] + 1 - min(raw_data.loc[:,"DAP"]))
    log_data.loc[:,"RTP"] = np.log(raw_data.loc[:,"RTP"] + 1 - min(raw_data.loc[:,"RTP"]))
    log_data.to_csv('nyiso/NYISO_'+zone+'_log.csv')

    # Split dataset: 2015 year for training and 2016-2017 years for testing
    x_train_df = log_data.iloc[8760*3:8760*4+24,:]
    x_test_df = log_data.iloc[8760*4+24:,:]

    y_train_df = log_data.iloc[8760*3:8760*4+24,2:]
    y_test_df = log_data.iloc[8760*4+24:,2:]

    # Standardization
    x_mean, x_std = x_train_df.mean(), x_train_df.std()
    y_mean, y_std = y_train_df.mean(), y_train_df.std()

    # Evaluation metrics
    print(zone)
    print('DAP mean: {:.4f}'.format(raw_data.iloc[8760*4+24:,:1].mean()[0]))
    print('DAP std: {:.4f}'.format(raw_data.iloc[8760*4+24:,:1].std()[0]))
    print('RTP mean: {:.4f}'.format(raw_data.iloc[8760*4+24:,2:].mean()[0]))
    print('RTP std: {:.4f}'.format(raw_data.iloc[8760*4+24:,2:].std()[0]))


CAPITL
44.6038
18.9331
44.1633
29.5993
CENTRL
29.8999
16.3021
29.5728
22.6937
DUNWOD
41.6494
18.1111
41.4403
25.9568
GENESE
28.7855
15.9150
28.5536
22.5367
HUD VL
40.7378
17.2660
40.0888
23.3379
LONGIL
54.0603
31.3705
54.7833
63.6448
MHK VL
30.1917
16.6750
30.0775
23.3890
MILLWD
41.6300
18.2120
41.7525
29.6173
N.Y.C.
42.5206
18.9206
42.4620
26.8635
NORTH
22.6753
16.0531
23.5312
30.2330
WEST
31.0270
19.4403
30.8867
27.0452
